### Manipulation of PEAKS de novo results of _T. weiss_  LC-MS/MS data using Python.

Starting with:

    PEAKS de novo results (.csv) of PTM-optimized sequencing >80% ALC
    from Thermo Fusion tribrid runs at the UW Proteomics Resource center, January 2021
    combined from multiple injections

Goal:

    Files with stripped (no PTMs) peptide lists and
    Columns with #'s of each modification in every sequence
    Column with stripped peptide lengths (# amino acids)
    
### To use:

#### 1. Change the input file name in *IN 4*
#### 2. Change output file name in *IN 6*, *IN 7*, *IN 8*

In [9]:
# LIBRARIES
#import pandas library for working with tabular data
import os
os.getcwd()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import kde
#import regular expresson (regex)
import re
#check pandas version
pd.__version__

'1.0.5'

In [10]:
cd /home/millieginty/Documents/git-repos/rot-mayer/data/MED_Weissrot_Fusion_UWPR2021/MED_Weissrot_Fusion_323-T2dig-all_DENOVO_76/

/home/millieginty/Documents/git-repos/rot-mayer/data/MED_Weissrot_Fusion_UWPR2021/MED_Weissrot_Fusion_323-T2dig-all_DENOVO_76


In [11]:
# read the CSVs into a dataframe using the read_csv function and call 'peaks'

peaks323 = pd.read_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/MED_Weissrot_Fusion_UWPR2021/MED_Weissrot_Fusion_323-T2dig-all_DENOVO_76/323-T2dig-all_DENOVO_76_DN.csv")

print("# redundant Peaks peptides >80% ALC in combined dataframe:", len(peaks323))

# look at the dataframe
peaks323.head()

# redundant Peaks peptides >80% ALC in combined dataframe: 5780


,Fraction,Scan,Source File,Peptide,Tag Length,ALC (%),length,m/z,z,RT,Area,Mass,ppm,PTM,local confidence (%),tag (>=0%),mode
0,6,4061,20210112_Weissrot_323_T2_trypsin_DDA_120min_2u...,TPEEEER,7,99,7,445.1986,2,17.90,169000.0,888.3824,0.3,NaN,100 100 100 100 100 100 97,TPEEEER,HCD
1,9,300,20210117_Weissrot_323_T2_trypsin_DDA_120min_2u...,HAVSEGTK,8,99,8,414.7150,2,1.99,7280.0,827.4137,2.0,NaN,99 99 100 99 100 97 100 99,HAVSEGTK,HCD
2,6,16101,20210112_Weissrot_323_T2_trypsin_DDA_120min_2u...,KQELEDLTK,9,98,9,552.3011,2,46.67,3600000.0,1102.5869,0.8,NaN,97 99 99 98 100 99 100 100 99,KQELEDLTK,HCD
3,9,3161,20210117_Weissrot_323_T2_trypsin_DDA_120min_2u...,TPEEEER,7,98,7,445.1991,2,16.05,125000.0,888.3824,1.4,NaN,100 100 100 99 99 99 95,TPEEEER,HCD
4,6,5401,20210112_Weissrot_323_T2_trypsin_DDA_120min_2u...,TTEEAVR,7,98,7,403.2061,2,19.46,419000.0,804.3977,0.0,NaN,100 100 100 99 98 98 98,TTEEAVR,HCD


The peptide column has the masses of modifications (e.g., 57.02 Da for carbamidomethylation of cysteine). We want to make new columns with all that information and make a column with only the 'stripped' peptide sequence that's just amino acids - this we can then align against other sequences, for example.

Modified residues were allowed for:

    fixed carbamodimethylation of cysteine 57.021464 C
    varialbe oxidation of methionine, lysine, proline, arginine, tyrosine: 15.9949 MKPRY
    variable deamidation of asparagine, glumatine: 0.984016 NQ
    variable methylation of lysine and arginine: 14.015650 KR
    variable pyro-glu from glutamine: -17.03 Q
    variable acetylation of lysine: 42.01 K


We'll then write this manipulated dataframe to a new file.

In [12]:
# use a count function to enumerate the # of A's (alanines) in each peptide
peaks323['A'] = peaks323['Peptide'].str.count("A")

# use a count function to enumerate the # of C's (cysteines) in each peptide
peaks323['C'] = peaks323['Peptide'].str.count("C")

# use a count function to enumerate the # of D's (aspartic acids) in each peptide
peaks323['D'] = peaks323['Peptide'].str.count("D")

# use a count function to enumerate the # of E's (glutamic acids) in each peptide
peaks323['E'] = peaks323['Peptide'].str.count("E")

# use a count function to enumerate the # of F's (phenylalanines) in each peptide
peaks323['F'] = peaks323['Peptide'].str.count("F")

# use a count function to enumerate the # of G's (glycines) in each peptide
peaks323['G'] = peaks323['Peptide'].str.count("G")

# use a count function to enumerate the # of H's (histidines) in each peptide
peaks323['H'] = peaks323['Peptide'].str.count("H")

# use a count function to enumerate the # of I's (isoleucines) in each peptide
# in peaks323 output, there will be no isoleucines (they're lumped in with leucines)
peaks323['I'] = peaks323['Peptide'].str.count("I")

# use a count function to enumerate the # of K's (lysines) in each peptide
peaks323['K'] = peaks323['Peptide'].str.count("K")

# use a count function to enumerate the # of L's (leucines) in each peptide
# also these include the isoleucines
peaks323['L'] = peaks323['Peptide'].str.count("L")

# use a count function to enumerate the # of M's (methionines) in each peptide
peaks323['M'] = peaks323['Peptide'].str.count("M")

# use a count function to enumerate the # of N's (asparagines) in each peptide
peaks323['N'] = peaks323['Peptide'].str.count("N")

# use a count function to enumerate the # of P's ([prolines]) in each peptide
peaks323['P'] = peaks323['Peptide'].str.count("P")

# use a count function to enumerate the # of Q's (glutamines) in each peptide
peaks323['Q'] = peaks323['Peptide'].str.count("Q")

# use a count function to enumerate the # of R's (arginines) in each peptide
peaks323['R'] = peaks323['Peptide'].str.count("R")

# use a count function to enumerate the # of S's (serines) in each peptide
peaks323['S'] = peaks323['Peptide'].str.count("S")

# use a count function to enumerate the # of T's (threonines) in each peptide
peaks323['T'] = peaks323['Peptide'].str.count("T")

# use a count function to enumerate the # of V's (valines) in each peptide
peaks323['V'] = peaks323['Peptide'].str.count("V")

# use a count function to enumerate the # of W's (tryptophans) in each peptide
peaks323['W'] = peaks323['Peptide'].str.count("W")

# use a count function to enumerate the # of Y's (tyrosines) in each peptide
peaks323['Y'] = peaks323['Peptide'].str.count("Y")

# use a count function to enumerate the # of carbamidomethylated C's in each peptide
peaks323['c-carb'] = peaks323['Peptide'].str.count("57.02")

# use a count function to enumerate the # of oxidized M's in each peptide
peaks323['m-oxid'] = peaks323['Peptide'].apply(lambda x: x.count('M(+15.99)'))

# use a count function to enumerate the # of oxidized K's in each peptide
peaks323['k-oxid'] = peaks323['Peptide'].apply(lambda x: x.count('K(+15.99)'))

# use a count function to enumerate the # of oxidized P's in each peptide
peaks323['p-oxid'] = peaks323['Peptide'].apply(lambda x: x.count('P(+15.99)'))

# use a count function to enumerate the # of oxidized R's in each peptide
peaks323['r-oxid'] = peaks323['Peptide'].apply(lambda x: x.count('R(+15.99)'))

# use a count function to enumerate the # of oxidized Y's in each peptide
peaks323['y-oxid'] = peaks323['Peptide'].apply(lambda x: x.count('Y(+15.99)'))

# use a lamba function to enumerate the # of deamidated N's in each peptide
peaks323['n-deam'] = peaks323['Peptide'].apply(lambda x: x.count('N(+.98)'))

# use a count function to enumerate the # of deamidated Q's in each peptide
#peaks323['q-deam'] = peaks323['Peptide'].apply(lambda x: x.count('Q(+.98)'))

# use a count function to enumerate the # of methylated K's in each peptide
peaks323['k-meth'] = peaks323['Peptide'].apply(lambda x: x.count('K(+14.02)'))

# use a count function to enumerate the # of methylated R's in each peptide
peaks323['r-meth'] = peaks323['Peptide'].apply(lambda x: x.count('R(+14.02)'))

# use a count function to enumerate the # of pyro glu Q's in each peptide
peaks323['q-pyro'] = peaks323['Peptide'].apply(lambda x: x.count('Q(-17.03)'))

# use a count function to enumerate the # of acetylation of K's in each peptide
peaks323['k-acet'] = peaks323['Peptide'].apply(lambda x: x.count('K(+42.01)'))

# create a column with 'stripped' peptide sequences using strip
peaks323['stripped peptide'] = peaks323['Peptide'].str.replace(r"\(.*\)","")

# add a column with the stripped peptide length (number of AAs)
peaks323['stripped length'] = peaks323['stripped peptide'].apply(len)

# total the number of modifications in sequence
peaks323['ptm-total'] = peaks323['c-carb'] + peaks323['m-oxid'] + peaks323['k-oxid'] + peaks323['p-oxid'] \
+ peaks323['r-oxid'] + peaks323['y-oxid'] + peaks323['n-deam'] + peaks323['k-meth'] + peaks323['r-meth'] \
+ peaks323['q-pyro'] + peaks323['k-acet']

# calculate NAAF numerator for each peptide k
peaks323['NAAF num.'] = peaks323['Area'] / peaks323['stripped length']

# write modified dataframe to new txt file, same name + 'stripped'
peaks323.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDN/TW_323_T2_trypsin_combine_PTMopt_DN80.csv")

# check out the results
peaks323.head()

,Fraction,Scan,Source File,Peptide,Tag Length,ALC (%),length,m/z,z,RT,...,y-oxid,n-deam,k-meth,r-meth,q-pyro,k-acet,stripped peptide,stripped length,ptm-total,NAAF num.
0,6,4061,20210112_Weissrot_323_T2_trypsin_DDA_120min_2u...,TPEEEER,7,99,7,445.1986,2,17.90,...,0,0,0,0,0,0,TPEEEER,7,0,24142.857143
1,9,300,20210117_Weissrot_323_T2_trypsin_DDA_120min_2u...,HAVSEGTK,8,99,8,414.7150,2,1.99,...,0,0,0,0,0,0,HAVSEGTK,8,0,910.000000
2,6,16101,20210112_Weissrot_323_T2_trypsin_DDA_120min_2u...,KQELEDLTK,9,98,9,552.3011,2,46.67,...,0,0,0,0,0,0,KQELEDLTK,9,0,400000.000000
3,9,3161,20210117_Weissrot_323_T2_trypsin_DDA_120min_2u...,TPEEEER,7,98,7,445.1991,2,16.05,...,0,0,0,0,0,0,TPEEEER,7,0,17857.142857
4,6,5401,20210112_Weissrot_323_T2_trypsin_DDA_120min_2u...,TTEEAVR,7,98,7,403.2061,2,19.46,...,0,0,0,0,0,0,TTEEAVR,7,0,59857.142857


In [13]:
# made a new dataframe that contains the sums of certain columns 
# in the stripped peptide dataframe above (for >80% ALC)

index = ['sample total']

data = {'A': peaks323['A'].sum(),
        'C': peaks323['C'].sum(),
        'D': peaks323['D'].sum(),
        'E': peaks323['E'].sum(),
        'F': peaks323['F'].sum(),
        'G': peaks323['G'].sum(),
        'H': peaks323['H'].sum(),
        'I': peaks323['I'].sum(),
        'K': peaks323['K'].sum(),
        'L': peaks323['L'].sum(),
        'M': peaks323['M'].sum(),
        'N': peaks323['N'].sum(),
        'P': peaks323['P'].sum(),
        'Q': peaks323['Q'].sum(),
        'R': peaks323['R'].sum(),
        'S': peaks323['S'].sum(),
        'T': peaks323['T'].sum(),
        'V': peaks323['V'].sum(),
        'W': peaks323['W'].sum(),
        'Y': peaks323['Y'].sum(),
        'c-carb': peaks323['c-carb'].sum(),
        'm-oxid': peaks323['m-oxid'].sum(),
        'k-oxid': peaks323['k-oxid'].sum(),
        'p-oxid': peaks323['p-oxid'].sum(),
        'r-oxid': peaks323['r-oxid'].sum(),
        'y-oxid': peaks323['y-oxid'].sum(),
        'n-deam': peaks323['n-deam'].sum(),
        'k-meth': peaks323['k-meth'].sum(),
        'r-meth': peaks323['r-meth'].sum(),
        'q-pyro': peaks323['q-pyro'].sum(),
        'k-acet': peaks323['k-acet'].sum(),
        'Total area': peaks323['Area'].sum(),
        'Total length': peaks323['stripped length'].sum()
       }

totalpeaks323 = pd.DataFrame(data, columns=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', \
                                            'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', \
                                            'c-carb', 'm-oxid', 'k-oxid', 'p-oxid', 'r-oxid', \
                                            'y-oxid', 'n-deam', 'k-meth', 'r-meth', 'q-pyro', \
                                            'k-acet', 'Total area', 'Total length'], index=index)

# calculate percentage of C's with carb (should be 1.0)
totalpeaks323['% C w/ carb'] = totalpeaks323['c-carb'] / totalpeaks323['C'] 

# calculate percentage of M's that are oxidized
totalpeaks323['% M w/ oxid'] = totalpeaks323['m-oxid'] / totalpeaks323['M'] 

# calculate percentage of K's that are oxidized
totalpeaks323['% K w/ oxid'] = totalpeaks323['k-oxid'] / totalpeaks323['K'] 

# calculate percentage of P's that are oxidized
totalpeaks323['% P w/ oxid'] = totalpeaks323['p-oxid'] / totalpeaks323['P'] 

# calculate percentage of R's that are oxidized
totalpeaks323['% R w/ oxid'] = totalpeaks323['p-oxid'] / totalpeaks323['R'] 

# calculate percentage of Y's that are oxidized
totalpeaks323['% Y w/ oxid'] = totalpeaks323['y-oxid'] / totalpeaks323['Y'] 

# calculate percentage of N's that are deamidated
totalpeaks323['% N w/ deam'] = totalpeaks323['n-deam'] / totalpeaks323['N'] 

# calculate percentage of K's that are methylated
totalpeaks323['% K w/ meth'] = totalpeaks323['k-meth'] / totalpeaks323['K'] 

# calculate percentage of R's that are methylated
totalpeaks323['% R w/ meth'] = totalpeaks323['r-meth'] / totalpeaks323['R'] 

# calculate percentage of Q's that are pyro glu'd
totalpeaks323['% Q w/ pyro'] = totalpeaks323['q-pyro'] / totalpeaks323['Q'] 

# calculate percentage of K's that are acetylation
totalpeaks323['% K w/ acet'] = totalpeaks323['k-acet'] / totalpeaks323['K'] 

# calculate NAAF denominator for all peptides in dataset i
totalpeaks323['NAAF denom.'] = totalpeaks323['Total area'] / totalpeaks323['Total length']

# write modified dataframe to new txt file
totalpeaks323.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDN/TW_323_T2_trypsin_combine_PTMopt_DN80_totals.csv")

totalpeaks323.head()

,A,C,D,E,F,G,H,I,K,L,...,% K w/ oxid,% P w/ oxid,% R w/ oxid,% Y w/ oxid,% N w/ deam,% K w/ meth,% R w/ meth,% Q w/ pyro,% K w/ acet,NAAF denom.
sample total,3563,502,2646,4830,1350,2423,360,0,4562,4945,...,0.072994,0.295097,0.311285,0.131765,0.340042,0.067953,0.059875,0.080576,0.11815,133504.991352


In [14]:
# use the calculated NAAF factor (in totalpeaks dataframe, above) to caluclate the NAAF 
# NAAF: normalized normalized area abundance factor

# don't have to worry here about DECOY hits messing with Area totals
# but we would with Comet results

NAAF80 = 133504.991352

# use NAAF >80% ALC to get NAAF factor
peaks323['NAAF factor'] = (peaks323['NAAF num.'])/NAAF80

# make a dataframe that contains only what we need: sequences, AAs, PTMs
peaksAAPTM_323 = peaks323[['stripped peptide', 'NAAF factor', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'K', 'I', 'L', \
                                'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y', 'c-carb', 'm-oxid', \
                                'k-oxid', 'p-oxid', 'r-oxid', 'y-oxid', 'n-deam', 'k-meth', 'r-meth', \
                                'q-pyro', 'k-acet']].copy()

# multiply the NAAF80 factor by the AA total to normalize its abundance by peak area and peptide length

peaksAAPTM_323['A-NAAF80'] = peaksAAPTM_323['A'] * peaks323['NAAF factor']
peaksAAPTM_323['C-NAAF80'] = peaksAAPTM_323['C'] * peaks323['NAAF factor']
peaksAAPTM_323['D-NAAF80'] = peaksAAPTM_323['D'] * peaks323['NAAF factor']
peaksAAPTM_323['E-NAAF80'] = peaksAAPTM_323['E'] * peaks323['NAAF factor']
peaksAAPTM_323['F-NAAF80'] = peaksAAPTM_323['F'] * peaks323['NAAF factor']
peaksAAPTM_323['G-NAAF80'] = peaksAAPTM_323['G'] * peaks323['NAAF factor']
peaksAAPTM_323['H-NAAF80'] = peaksAAPTM_323['H'] * peaks323['NAAF factor']
peaksAAPTM_323['I-NAAF80'] = peaksAAPTM_323['I'] * peaks323['NAAF factor']
peaksAAPTM_323['K-NAAF80'] = peaksAAPTM_323['K'] * peaks323['NAAF factor']
peaksAAPTM_323['L-NAAF80'] = peaksAAPTM_323['L'] * peaks323['NAAF factor']
peaksAAPTM_323['M-NAAF80'] = peaksAAPTM_323['M'] * peaks323['NAAF factor']
peaksAAPTM_323['N-NAAF80'] = peaksAAPTM_323['N'] * peaks323['NAAF factor']
peaksAAPTM_323['P-NAAF80'] = peaksAAPTM_323['P'] * peaks323['NAAF factor']
peaksAAPTM_323['Q-NAAF80'] = peaksAAPTM_323['Q'] * peaks323['NAAF factor']
peaksAAPTM_323['R-NAAF80'] = peaksAAPTM_323['R'] * peaks323['NAAF factor']
peaksAAPTM_323['S-NAAF80'] = peaksAAPTM_323['S'] * peaks323['NAAF factor']
peaksAAPTM_323['T-NAAF80'] = peaksAAPTM_323['T'] * peaks323['NAAF factor']
peaksAAPTM_323['V-NAAF80'] = peaksAAPTM_323['V'] * peaks323['NAAF factor']
peaksAAPTM_323['W-NAAF80'] = peaksAAPTM_323['W'] * peaks323['NAAF factor']
peaksAAPTM_323['Y-NAAF80'] = peaksAAPTM_323['Y'] * peaks323['NAAF factor']

# multiply the NAAF80 factor by the PTMs normalize its abundance by peak area and peptide length

peaksAAPTM_323['ccarb-NAAF80'] = peaksAAPTM_323['c-carb'] * peaksAAPTM_323['NAAF factor']
peaksAAPTM_323['moxid-NAAF80'] = peaksAAPTM_323['m-oxid'] * peaksAAPTM_323['NAAF factor']
peaksAAPTM_323['koxid-NAAF80'] = peaksAAPTM_323['k-oxid'] * peaksAAPTM_323['NAAF factor']
peaksAAPTM_323['poxid-NAAF80'] = peaksAAPTM_323['p-oxid'] * peaksAAPTM_323['NAAF factor']
peaksAAPTM_323['roxid-NAAF80'] = peaksAAPTM_323['r-oxid'] * peaksAAPTM_323['NAAF factor']
peaksAAPTM_323['yoxid-NAAF80'] = peaksAAPTM_323['y-oxid'] * peaksAAPTM_323['NAAF factor']
peaksAAPTM_323['ndeam-NAAF80'] = peaksAAPTM_323['n-deam'] * peaksAAPTM_323['NAAF factor']
peaksAAPTM_323['kmeth-NAAF80'] = peaksAAPTM_323['k-meth'] * peaksAAPTM_323['NAAF factor']
peaksAAPTM_323['rmeth-NAAF80'] = peaksAAPTM_323['r-meth'] * peaksAAPTM_323['NAAF factor']
peaksAAPTM_323['qpyro-NAAF80'] = peaksAAPTM_323['q-pyro'] * peaksAAPTM_323['NAAF factor']
peaksAAPTM_323['kacet-NAAF80'] = peaksAAPTM_323['k-acet'] * peaksAAPTM_323['NAAF factor']

# write the dataframe to a new csv
peaksAAPTM_323.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDN/TW_323_T2_trypsin_combine_PTMopt_DN80_NAAF.csv")

peaksAAPTM_323.head()

,stripped peptide,NAAF factor,A,C,D,E,F,G,H,K,...,moxid-NAAF80,koxid-NAAF80,poxid-NAAF80,roxid-NAAF80,yoxid-NAAF80,ndeam-NAAF80,kmeth-NAAF80,rmeth-NAAF80,qpyro-NAAF80,kacet-NAAF80
0,TPEEEER,0.180839,0,0,0,4,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,HAVSEGTK,0.006816,1,0,0,1,0,1,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,KQELEDLTK,2.996143,0,0,1,2,0,0,0,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,TPEEEER,0.133756,0,0,0,4,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,TTEEAVR,0.448351,1,0,0,2,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# made a dataframe that's the sum of NAAF corrected AAs and PTMs

index = ['sample total']

data = {'NAAF': peaksAAPTM_323['NAAF factor'].sum(),
        'A': peaksAAPTM_323['A-NAAF80'].sum(),
        'C': peaksAAPTM_323['C-NAAF80'].sum(),
        'D': peaksAAPTM_323['D-NAAF80'].sum(),
        'E': peaksAAPTM_323['E-NAAF80'].sum(),
        'F': peaksAAPTM_323['F-NAAF80'].sum(),
        'G': peaksAAPTM_323['G-NAAF80'].sum(),
        'H': peaksAAPTM_323['H-NAAF80'].sum(),
        'I': peaksAAPTM_323['I-NAAF80'].sum(),
        'K': peaksAAPTM_323['K-NAAF80'].sum(),
        'L': peaksAAPTM_323['L-NAAF80'].sum(),
        'M': peaksAAPTM_323['M-NAAF80'].sum(),
        'N': peaksAAPTM_323['N-NAAF80'].sum(),
        'P': peaksAAPTM_323['P-NAAF80'].sum(),
        'Q': peaksAAPTM_323['Q-NAAF80'].sum(),
        'R': peaksAAPTM_323['R-NAAF80'].sum(),
        'S': peaksAAPTM_323['S-NAAF80'].sum(),
        'T': peaksAAPTM_323['T-NAAF80'].sum(),
        'V': peaksAAPTM_323['V-NAAF80'].sum(),
        'W': peaksAAPTM_323['W-NAAF80'].sum(),
        'Y': peaksAAPTM_323['Y-NAAF80'].sum(),
        'c-carb': peaksAAPTM_323['ccarb-NAAF80'].sum(),
        'm-oxid': peaksAAPTM_323['moxid-NAAF80'].sum(),
        'k-oxid': peaksAAPTM_323['koxid-NAAF80'].sum(),
        'p-oxid': peaksAAPTM_323['poxid-NAAF80'].sum(),
        'r-oxid': peaksAAPTM_323['roxid-NAAF80'].sum(),
        'y-oxid': peaksAAPTM_323['yoxid-NAAF80'].sum(),
        'n-deam': peaksAAPTM_323['ndeam-NAAF80'].sum(),
        'k-meth': peaksAAPTM_323['kmeth-NAAF80'].sum(),
        'r-meth': peaksAAPTM_323['rmeth-NAAF80'].sum(),
        'q-pyro': peaksAAPTM_323['qpyro-NAAF80'].sum(),
        'k-acet': peaksAAPTM_323['kacet-NAAF80'].sum()
       }

totalpeaks80_NAAF = pd.DataFrame(data, columns=['NAAF', 'A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', \
                                           'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', \
                                           'W', 'Y', 'c-carb', 'm-oxid', 'k-oxid', 'p-oxid', \
                                           'r-oxid', 'y-oxid', 'n-deam', 'k-meth', 'r-meth', \
                                            'q-pyro', 'k-acet'], index=index)

# calculate NAAF-corrected percentage of C's with carb (should be 1.0)
totalpeaks80_NAAF['% C w/ carb.'] = totalpeaks80_NAAF['c-carb'] / totalpeaks80_NAAF['C'] 

# calculate NAAF-corrected percentage of M's that are oxidized
totalpeaks80_NAAF['% M w/ oxid'] = totalpeaks80_NAAF['m-oxid'] / totalpeaks80_NAAF['M'] 

# calculate NAAF-corrected percentage of K's that are oxidized
totalpeaks80_NAAF['% K w/ oxid'] = totalpeaks80_NAAF['k-oxid'] / totalpeaks80_NAAF['K'] 

# calculate NAAF-corrected percentage of P's that are oxidized
totalpeaks80_NAAF['% P w/ oxid'] = totalpeaks80_NAAF['p-oxid'] / totalpeaks80_NAAF['P'] 

# calculate NAAF-corrected percentage of R's that are oxidized
totalpeaks80_NAAF['% R w/ oxid'] = totalpeaks80_NAAF['r-oxid'] / totalpeaks80_NAAF['R'] 

# calculate NAAF-corrected percentage of Y's that are oxidized
totalpeaks80_NAAF['% Y w/ oxid'] = totalpeaks80_NAAF['y-oxid'] / totalpeaks80_NAAF['Y'] 

# calculate NAAF-corrected percentage of N's that are deamidated
totalpeaks80_NAAF['% N w/ deam'] = totalpeaks80_NAAF['n-deam'] / totalpeaks80_NAAF['N'] 

# calculate NAAF-corrected percentage of K's that are methylated
totalpeaks80_NAAF['% K w/ meth'] = totalpeaks80_NAAF['k-meth'] / totalpeaks80_NAAF['K'] 

# calculate NAAF-corrected percentage of R's that are methylated
totalpeaks80_NAAF['% R w/ meth'] = totalpeaks80_NAAF['r-meth'] / totalpeaks80_NAAF['R'] 

# calculate NAAF-corrected percentage of Q's that are pyro glu'd
totalpeaks80_NAAF['% Q w/ pyro'] = totalpeaks80_NAAF['q-pyro'] / totalpeaks80_NAAF['Q'] 

# calculate NAAF-corrected percentage of K's that are methylated
totalpeaks80_NAAF['% K w/ acet'] = totalpeaks80_NAAF['k-acet'] / totalpeaks80_NAAF['K'] 

# calculate NAAF summed numerator over denominator (in above cell) for all peptides in dataset i: a check
totalpeaks80_NAAF['NAAF check'] = totalpeaks80_NAAF['NAAF'] / 133504.991352

# write modified dataframe to new txt file, same name + totals
totalpeaks80_NAAF.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDN/TW_323_T2_trypsin_combine_PTMopt_DN80_NAAF_totals.csv")

totalpeaks80_NAAF.head()

,NAAF,A,C,D,E,F,G,H,I,K,...,% K w/ oxid,% P w/ oxid,% R w/ oxid,% Y w/ oxid,% N w/ deam,% K w/ meth,% R w/ meth,% Q w/ pyro,% K w/ acet,NAAF check
sample total,6001.990934,2973.218088,588.20847,1637.271117,4223.900556,969.088984,2101.846385,429.078359,0.0,3849.593151,...,0.120515,0.199046,0.106455,0.180812,0.354271,0.090449,0.036714,0.188677,0.161752,0.044957


## Export stripped peptides >80% ALC

In [17]:
# keep only stripped peptide column 
pep80 = peaks323[["stripped peptide"]]

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing

pep80.to_csv("/home/millieginty/Documents/git-repos/TW_323_T2_trypsin_combine_PTMopt_DN80_stripped_peptides.txt", header=False, index=False)

# removing redundancy
peaks80dedup = pd.DataFrame.drop_duplicates(pep80)

# write altered dataframe to new txt file
# used header and index parameters to get rid of 'Peptide' header and the indexing

peaks80dedup.to_csv("/home/millieginty/Documents/git-repos/rot-mayer/data/processed/PeaksDN/TW_323_T2_trypsin_combine_PTMopt_DN80_nonredundant_stripped_peptides.txt", header=False, index=False)

print("# redundant stripped Peaks peptides >80% ALC", len(pep80))
print("# nonredundant stripped Peaks peptides >80% ALC", len(peaks80dedup))
print("average peptide length Peaks peptides >80% ALC", peaks323['stripped length'].mean())

# count all unique peptide (modified peptides included)
# keep only peptide column >80% ALC
pep80m = peaks323[["Peptide"]]

# deduplicate
pep80mdedup = pd.DataFrame.drop_duplicates(pep80m)

print("# redundant Peaks peptides >80% ALC", len(pep80m))
print("# nonredundant Peaks peptides", len(pep80mdedup))

# check
pep80.head()

# redundant stripped Peaks peptides >80% ALC 5780
# nonredundant stripped Peaks peptides >80% ALC 4613
average peptide length Peaks peptides >80% ALC 7.340484429065744
# redundant Peaks peptides >80% ALC 5780
# nonredundant Peaks peptides 4848


,stripped peptide
0,TPEEEER
1,HAVSEGTK
2,KQELEDLTK
3,TPEEEER
4,TTEEAVR
